In [1]:
import operator
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import neighbors, metrics

In [2]:
_BLANC = 'winequality-white.csv'

In [3]:
# Récuparation du dataset
data = pd.read_csv(_BLANC, sep=";")

In [4]:
def ma_fonction(p, nb_fold):
    """
    TBD
    """

    # Partie de découpage
    taille_fold = int(len(data)/nb_fold)

    somme = 0

    for i in range(0, nb_fold):
        # On gére d'abord les deux particuliers
        # Pour le premier découpage
        if i == 0:
            data_test = data[:(i+1)*taille_fold]
            data_train = data[(i+1)*taille_fold:]

        # Pour le dernier découpage
        elif i == (nb_fold-1):
            data_test = data[i*taille_fold:]
            data_train = data[:i*taille_fold]

        # Cas  général : On merge les deux bouts si on a extrait au départ un bout du milieu
        elif i != (nb_fold-1):
            data_test = data[i*taille_fold:(i+1)*taille_fold]
            data_train1 = data[:i*taille_fold]
            data_train2 = data[(i+1)*taille_fold:]

            frames = [data_train1, data_train2]
            data_train = pd.concat(frames)

        # Mise en forme du dataset
        x_train = data_train.as_matrix(data_train.columns[:-1])
        y_train = data_train.as_matrix([data_train.columns[-1]])

        # Mise en forme du dataset
        x_test = data_test.as_matrix(data_test.columns[:-1])
        y_test = data_test.as_matrix([data_test.columns[-1]])
        
        # Classification en binaire
        y_test = np.where(y_test < 6, 0, 1)

        # Scale
        std_scale = preprocessing.StandardScaler().fit(x_train)
        x_train_std = std_scale.transform(x_train)
        x_test_std = std_scale.transform(x_test)

        # Prédictions
        nbrs = neighbors.KNeighborsClassifier(n_neighbors=p).fit(x_train_std, y_train)
        y_pred = nbrs.predict(x_test_std)

        # Classification en binaire
        y_pred = np.where(y_pred < 6, 0, 1)

        # Cumul pour ensuite diviser et avoir la moyenne du score
        somme = somme + metrics.accuracy_score(y_test, y_pred)

    # Score
    return somme/nb_fold

In [5]:
# Fixer les valeurs des hyperparamètres à tester
param_grid = [3, 5, 7, 9, 11, 13, 15]

liste_finale = dict()

for p in param_grid:
    liste_finale[p] = (ma_fonction(p, 5))

print(liste_finale.items())

indicemax = max(liste_finale.items(), key=operator.itemgetter(1))[0]

print('Configuration optimale pour', indicemax, 'neighbors :', round(liste_finale[indicemax], 3))

C:\Users\Toni\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


dict_items([(3, 0.68372315572022657), (5, 0.69862197803569459), (7, 0.70718655929301477), (9, 0.71065574623093108), (11, 0.71821572784066201), (13, 0.71821760015311353), (15, 0.72148998624890537)])
Configuration optimale pour 15 neighbors : 0.721
